# Conway's Game of Life
---

The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, alive or dead. Every cell interacts with its eight neighbours, which are the cells that are horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

* Any live cell with fewer than two live neighbours dies, as if caused by under-population.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by over-population.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

The initial pattern constitutes the seed of the system. The first generation is created by applying the above rules simultaneously to every cell in the seed—births and deaths occur simultaneously, and the discrete moment at which this happens is sometimes called a tick (in other words, each generation is a pure function of the preceding one). The rules continue to be applied repeatedly to create further generations. [1]

## Example

Some images of Game of Life systems should go here.

## Administrivia

In [192]:
import numpy as np

## Design Decisions

We'll start by making some decisions about how to structure our program and the sorts of data types we'll use in our code.

> The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, alive or dead.

suggests that a two-dimensional array is a good structure for our Game of Life 'board'. 

## Intro to NumPy Arrays
TODO

## Prototyping Code

We're working in the realm of applications and not theory, so let's ignore the bit about the board being 'infinite'. Let's start by writing a function that will return a Game of Life board of a certain size. We'll use the number 0 to represent a dead cell, and 1 to represent a living cell, and to start with, let's return a board with only dead cells.

In [ ]:
def create_empty_board(height, width):
    """
    This function creates an empty Game of Life game board.
    
    Parameters
    ----------
        height : integer
            the height of the board
        width : integer
            the width of the board
            
    Returns
    ----------
        board : 2d array
            a Game of Life board, with 'living' and 'dead' cells
            
    Examples
    ----------
    >>> create_empty_board(3, 3)
    array([[ 0.,  0.,  0.],
           [ 0.,  0.,  0.],
           [ 0.,  0.,  0.]])
    """
    board = np.zeros((height, width))
    return board

In [ ]:
board = create_empty_board(3, 3)
board

So now we have a Game of Life board, but all the cells are dead so nothing interesting will happen. Let's create a better function to create a board in which some cells are alive and some are dead. We can use the NumPy `random.choice` function to accomplish this.

Anytime you are confused about how to use a function, in Jupyter Notebook you can write the `functionname` followed by a `?` and a help box will pop up.

In [ ]:
# np.random.choice?

In [ ]:
def create_board(height, width):
    """
    This function creates a Game of Life game board.
    
    Parameters
    ----------
        height : integer
            the height of the board
        width : integer
            the width of the board
            
    Returns
    ----------
        board : 2d array
            a Game of Life board, with 'living' and 'dead' cells
            
    Examples
    ----------
    >>> np.random.seed(1)
    >>> create_board(3, 3)
    array([[1, 1, 0],
           [0, 1, 1],
           [1, 1, 1]])
    """
    dead, alive = 0, 1
    board = np.random.choice([dead, alive], size=(height, width))
    return board

In [ ]:
board = create_board(3,3)
board

The way that this function currently works, each cell has a 50% chance of being alive and a 50% chance of being dead. One of the rules of Conway's Game of Life:

> Any live cell with more than three live neighbours dies, as if by over-population.

means that our game board has *way* too many living cells in it. We should add to the function so that we can control how likely a cell is to start as alive or dead.

In [ ]:
# Similar to the last function but we add the `p_alive` parameter.
def create_board(height, width, p_alive):
    """
    This function creates a Game of Life game board.
    
    Parameters
    ----------
        height : integer
            the height of the board
        width : integer
            the width of the board
        p_alive : float
            the chance that a cell is set to 'alive', must be between [0, 1]
            
    Returns
    ----------
        board : 2d array
            a Game of Life board, with 'living' and 'dead' cells
    
    Examples
    ----------
    >>> np.random.seed(1)
    >>> create_board(3, 3, .3)
    array([[0, 1, 0],           
           [0, 0, 0],
           [0, 0, 0]])
    """
    dead, alive = 0, 1
    p_dead = 1 - p_alive # We know that if a cell is not alive, it must be dead
    board = np.random.choice([dead, alive], p=[p_dead, p_alive], size=(height, width))
    
    return board

In [ ]:
board = create_board(3, 3, p_alive=0)
board

In [ ]:
board = create_board(3, 3, p_alive=.2)
board

In [ ]:
board = create_board(3, 3, p_alive=1)
board

Now we're ready to start building the actual simulation. A reminder of the rules of the Game of Life:

* Any live cell with fewer than two live neighbours dies, as if caused by under-population.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by over-population.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

Let's sketch out some pseudocode for the algorithm that we'll use to do this.

    Create a copy of the current board for the next board
    For each cell in the current board:
        Count the number of living neighbors it has.
        If (the number of living neighbors < 2) and (the cell is alive):
            It will be dead in the next board
        If (the number of living neighbors = 2 or 3) and (the cell is alive):
            It will be alive in the next board
        If (the number of living neighbors > 3) and the cell is alive:
            It will be dead in the next board
        If (the number of living neighbors = 3) and (the cell is dead):
            It will be alive in the next board
        Set the cell's value in the next board to either alive or dead.
        
Now that we have a pseudocode skeleton, we can sketch out some real code.

* TEACHING NOTE: Start by writing `step_forward` and everytime we write a function that doesn't exist yet, define it and document it outside of `step_forward` with a single pass statement.

* TEACHING NOTE: How to handle writing this code? I'm concerned about jumping back and forth, up and down. Maybe we could have two separate Jupyter Notebooks open?

In [ ]:
def count_living_neighbors(board, r, c):
    """
    Counts the number of living neighbors around an array cell.
    
    Parameters
    ----------
        board : 2d array
        r : int
            row index
        c : int
            column index
            
    Returns
    ----------
        int
            number of living neighbors

    """
    u_ = board[r-1, c]
    d_ = board[r+1, c]
    l_ = board[r, c-1]
    r_ = board[r, c+1]
    ul = board[r-1, c-1]
    ur = board[r-1, c+1]
    dl = board[r+1, c-1]
    dr = board[r+1, c+1]
    
    return (u_ + d_ + l_ + r_ + ul + ur + dl + dr)

In [ ]:
board = np.array([
    [0, 1, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [1, 0, 0, 0]
])

In [ ]:
count_living_neighbors(board, 1, 1) == 2

In [ ]:
count_living_neighbors(board, 1, 2) == 4

In [ ]:
count_living_neighbors(board, 0, 3) == 1

We're getting the error:

    IndexError: index 4 is out of bounds for axis 1 with size 4
    
How do we fix this? Let's remember that Python uses 0-based indexing, so an array axis with size 4 has indices 0, 1, 2, and 3. Trying to access the array entry at position 4 raises an IndexError. 

* NOTE: Python supports the indices -1, -2, -3, and -4 for an array with size 4. These indicate working backwards from the 0th entry in the array, so -1 is the last element, -2 is the second-to-last, etc... This is why we don't have issues accessing the -1st row when we look at the `ul` and `ur` elements in the above code.

A good question to ask is, what behavior do we want our code to have when it tries to access this index? We've decided that we want our simulation board to 'wrap around' so the element after the last one in a given row/column should be the *first* one in that row/column. In this specific case, for example, instead of index 4 we should pass in index 0.

* TEACHING NOTE: Is the modulo operator too much here?

Luckily, there is an operator just for this situation! Enter the % (modulo).

## Modulo

> The modulo operation finds the remainder after division of one number by another. [2]

In [ ]:
1 % 1

In [ ]:
2 % 1

In [ ]:
5 % 2

In [ ]:
array = [0, 1, 2, 3]
print('Length:', len(array))

In [ ]:
i = 4
i %= len(array)
i

In [ ]:
board.shape

## Back to Code

In [ ]:
def count_living_neighbors(board, r, c):
    """
    Counts the number of living neighbors around an array cell.
    
    Parameters
    ----------
        board : 2d array
        r : int
            row index
        c : int
            column index
            
    Returns
    ----------
        int
            number of living neighbors

    """
    num_rows, num_cols = board.shape
    
    u_ = board[(r-1) % num_rows, c % num_cols]
    d_ = board[(r+1) % num_rows, c % num_cols]
    l_ = board[r % num_rows, (c-1) % num_cols]
    r_ = board[r % num_rows, (c+1) % num_cols]
    ul = board[(r-1) % num_rows, (c-1) % num_cols]
    ur = board[(r-1) % num_rows, (c+1) % num_cols]
    dl = board[(r+1) % num_rows, (c-1) % num_cols]
    dr = board[(r+1) % num_rows, (c+1) % num_cols]
    
    return (u_ + d_ + l_ + r_ + ul + ur + dl + dr)

def compute_next_state(current_state, num_living_neighbors):
    """
    Computes the state of a cell in the next generation of a Game of Life.
    
    Parameters
    ----------
        current_state : int
        num_living_neighbors : int
        
    Returns
    ----------
        next_state : int
        
    """
    dead, alive = 0, 1
    next_state = dead
    
    if (num_living_neighbors < 2) and (current_state == alive):
        # Dies by under-population
        next_state = dead
    elif ( 2 <= num_living_neighbors <= 3) and (current_state == alive):
        # Survives on to the next generation
        next_state = alive
    elif (num_living_neighbors > 3) and (current_state == alive):
        # Dies by over-population
        next_state = dead
    elif (num_living_neighbors == 3) and (current_state == dead):
        # Live cell is birthed 
        next_state = alive
    
    return next_state

def step_forward(board):
    """
    Computes the next generation of a Game of Life board.
    
    Parameters
    ----------
        board : 2d array
        
    Returns
    ----------
        next_board : 2d array
    """
    # Copy the current board for the next generation
    next_board = board.copy()
    # Loop over all of the cells in the current board
    rows, cols = board.shape
    for r in range(rows):
        for c in range(cols):
            # Fetch the current state of this cell
            current_state = board[r, c]
            # Count the number of living neighbors
            num_living_neighbors = count_living_neighbors(board, r, c)
            # Figure out if this cell will be alive or dead in the next board
            next_state = compute_next_state(current_state, num_living_neighbors)
#             next_state = 1
            # Set the state of this cell in the next board
            next_board[r, c] = next_state  
    
    return next_board

In [ ]:
board = create_board(5, 5, .3)
print('Starting:')
print(board)
print('Next Step:')
board = step_forward(board)
print(board)

## Encapsulating in a Class

We've been writing code in somewhat of a 'functional' style --- that is we've been writing separate functions that each accomplish one particular task and are isolated from each other. This is a perfectly valid way of writing code, and has pros and cons just like any other style of progoramming. What we'll explore next is a style of programming called Object Oriented Programming (OOP). OOP is a programming paradigm that gives us the ability to think about and structure our code as a real-world object. So instead of passing arrays between functions, we'll create a 'Game of Life board' object with persistent data and we'll use that for our simulation (BAD). This style of programming also has pros and cons. We won't get into those here, I think that the best way to learn about which programming paradigm you should choose to use for a given task is through experience. 

### A Quick Introduction to Classes

In [ ]:
# Simplest possible class
class MyClass:
    attribute = 'This is an attribute!'
    
MyClass.attribute

In [ ]:
# A bit more complicated
class MyClass:
    def __init__(self, attribute):
        self.attribute = attribute
        
instance = MyClass('This is an instance attribute')
instance.attribute

### Game of Life Class

Let's start by thinking about the properties that our Game of Life object should have and the sorts of actions it should be able to perform.

* It should be able to start a simulation with a new 'board'
* It should be able to step forward through generations
* We may want to start a new simulation if all of the cells in the board are dead, so we should write a function to query the board to see if all the cells are dead

In [ ]:
# Skeleton Class
class GameOfLife:
    """Conway's Game of Life"""
    def __init__(self):
        """Initialize a class instance"""
        pass

    def start(self):
        """Start a new simulation."""
        pass
    
    def step(self):
        """Step the simulation forward a generation."""
        pass
    
    def is_empty(self):
        """Check to see if the Game of Life board is empty"""
        pass
    
    def restart(self):
        """Restart the Game of Life simulation with a new board"""
        pass

Now let's start filling this class in.

We'll start with our initialization method. We don't have a simulation running yet, but we want a 'board' attribute to be accessible to the other functions in the class, so we'll go ahead and add an empty board.

In [186]:
class GameOfLife:
    """Conway's Game of Life"""
    def __init__(self):
        """Initialize a class instance"""
        self.board = None

    def start(self):
        """Start a new simulation."""
        pass
    
    def step(self):
        """Step the simulation forward a generation."""
        pass
    
    def is_empty(self):
        """Check to see if the Game of Life board is empty"""
        pass
    
    def restart(self):
        """Restart the Game of Life simulation with a new board"""
        pass

In [ ]:
simulation = GameOfLife()
print(simulation.board)

Next we'll write the start method. This should create a new game board of a given size. We'll rip off the `create_board` function that we wrote earlier, rename it as `new_board`, which makes more sense in this context, and call that function under our class's start method.

In [193]:
class GameOfLife:
    DEAD, ALIVE = 0, 1 # These are Class variables
    """Conway's Game of Life"""
    def __init__(self):
        """Initialize a class instance"""
        self.board = None

    def start(self, height, width, p_alive=.3):
        """Start a new simulation."""
        self.new_board(height, width, p_alive)
    
    def step(self):
        """Step the simulation forward a generation."""
        pass
    
    def is_empty(self):
        """Check to see if the Game of Life board is empty"""
        pass
    
    def new_board(self, height, width, p_alive):
        """
        This function creates a Game of Life game board.

        Parameters
        ----------
            height : integer
                the height of the board
            width : integer
                the width of the board
            p_alive : float
                the chance that a cell is set to 'alive', must be between [0, 1]
                
        """
        p_dead = 1 - p_alive # We know that if a cell is not alive, it must be dead
        self.board = np.random.choice(
            [self.DEAD, self.ALIVE], p=[p_dead, p_alive], size=(height, width)
        )
    
    def restart(self):
        """Restart the Game of Life simulation with a new board"""
        pass

In [ ]:
simulation = GameOfLife()
simulation.start(height=10, width=10)
simulation.board

Next, let's add in the functionality to step to the next generation.

In [194]:
class GameOfLife:
    """Conway's Game of Life"""
    DEAD, ALIVE = 0, 1
    def __init__(self):
        """Initialize a class instance"""
        self.board = None

    def start(self, height, width, p_alive=.3):
        """Start a new simulation."""
        self.new_board(height, width, p_alive)
    
    def step(self):
        """Step the simulation forward a generation."""
        # Copy the current board for the next generation
        next_board = self.board.copy()
        # Loop over all of the cells in the current board
        num_rows, num_cols = self.board.shape
        for r in range(num_rows):
            for c in range(num_cols):
                # Fetch the current state of this cell
                current_state = self.board[r, c]
                # Count the number of living neighbors
                num_living_neighbors = self.count_living_neighbors(r, c)
                # Figure out if this cell will be alive or dead in the next board
                next_state = self.compute_next_state(current_state, num_living_neighbors)
    #             next_state = 1
                # Set the state of this cell in the next board
                next_board[r, c] = next_state  

        self.board = next_board
    
    def is_empty(self):
        """Check to see if the Game of Life board is empty"""
        pass
    
    def new_board(self, height, width, p_alive):
        """
        This function creates a Game of Life game board.

        Parameters
        ----------
            height : integer
                the height of the board
            width : integer
                the width of the board
            p_alive : float
                the chance that a cell is set to 'alive', must be between [0, 1]
                
        """
        dead, alive = 0, 1
        p_dead = 1 - p_alive # We know that if a cell is not alive, it must be dead
        self.board = np.random.choice(
            [self.DEAD, self.ALIVE], p=[p_dead, p_alive], size=(height, width)
        )

    def count_living_neighbors(self, r, c):
        """
        Counts the number of living neighbors around an array cell.

        Parameters
        ----------
            board : 2d array
            r : int
                row index
            c : int
                column index

        Returns
        ----------
            int
                number of living neighbors

        """
        num_rows, num_cols = self.board.shape

        u_ = self.board[(r-1) % num_rows, c % num_cols]
        d_ = self.board[(r+1) % num_rows, c % num_cols]
        l_ = self.board[r % num_rows, (c-1) % num_cols]
        r_ = self.board[r % num_rows, (c+1) % num_cols]
        ul = self.board[(r-1) % num_rows, (c-1) % num_cols]
        ur = self.board[(r-1) % num_rows, (c+1) % num_cols]
        dl = self.board[(r+1) % num_rows, (c-1) % num_cols]
        dr = self.board[(r+1) % num_rows, (c+1) % num_cols]

        return (u_ + d_ + l_ + r_ + ul + ur + dl + dr)

    def compute_next_state(self, current_state, num_living_neighbors):
        """
        Computes the state of a cell in the next generation of a Game of Life.

        Parameters
        ----------
            current_state : int
            num_living_neighbors : int

        Returns
        ----------
            next_state : int

        """
        next_state = self.DEAD

        if (num_living_neighbors < 2) and (current_state == self.ALIVE):
            # Dies by under-population
            next_state = self.DEAD
        elif ( 2 <= num_living_neighbors <= 3) and (current_state == self.ALIVE):
            # Survives on to the next generation
            next_state = self.ALIVE
        elif (num_living_neighbors > 3) and (current_state == self.ALIVE):
            # Dies by over-population
            next_state = self.DEAD
        elif (num_living_neighbors == 3) and (current_state == self.DEAD):
            # Live cell is birthed 
            next_state = self.ALIVE

        return next_state
    
    def restart(self):
        """Restart the Game of Life simulation with a new board"""
        pass

In [195]:
simulation = GameOfLife()
simulation.start(height=10, width=10)
print('Starting Board:')
print(simulation.board)
print('Next Board:')
simulation.step()
print(simulation.board)

Starting Board:
[[0 0 1 1 1 1 0 1 0 0]
 [0 0 1 0 1 1 0 0 0 1]
 [0 1 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 1 0 0]
 [0 1 0 1 0 0 0 0 0 0]
 [1 0 0 1 0 1 0 0 0 1]
 [1 1 0 0 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 0 1]
 [1 0 0 1 1 1 0 1 0 0]
 [1 1 0 1 1 0 0 0 0 0]]
Next Board:
[[1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 1 0]
 [0 0 1 1 1 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0]]


And finally let's add the ability to query the board to see if it is empty and to restart, if it is.

In [197]:
class GameOfLife:
    """Conway's Game of Life"""
    DEAD, ALIVE = 0, 1
    def __init__(self):
        """Initialize a class instance"""
        self.board = None
        
    def is_empty(self):
        """Check to see if the Game of Life board is empty"""
        return (self.board == 0).all()
    
    def restart(self, height, width, p_alive):
        """Restart the Game of Life simulation with a new board"""
        self.board = self.new_board(height, width, p_alive)
    
    def start(self, height, width, p_alive=.3):
        """Start a new simulation."""
        self.new_board(height, width, p_alive)
    
    def step(self):
        """Step the simulation forward a generation."""
        # Copy the current board for the next generation
        next_board = self.board.copy()
        # Loop over all of the cells in the current board
        num_rows, num_cols = self.board.shape
        for r in range(num_rows):
            for c in range(num_cols):
                # Fetch the current state of this cell
                current_state = self.board[r, c]
                # Count the number of living neighbors
                num_living_neighbors = self.count_living_neighbors(r, c)
                # Figure out if this cell will be alive or dead in the next board
                next_state = self.compute_next_state(current_state, num_living_neighbors)
    #             next_state = 1
                # Set the state of this cell in the next board
                next_board[r, c] = next_state  

        self.board = next_board
    
    def is_empty(self):
        """Check to see if the Game of Life board is empty"""
        pass
    
    def new_board(self, height, width, p_alive):
        """
        This function creates a Game of Life game board.

        Parameters
        ----------
            height : integer
                the height of the board
            width : integer
                the width of the board
            p_alive : float
                the chance that a cell is set to 'alive', must be between [0, 1]
                
        """
        dead, alive = 0, 1
        p_dead = 1 - p_alive # We know that if a cell is not alive, it must be dead
        self.board = np.random.choice(
            [self.DEAD, self.ALIVE], p=[p_dead, p_alive], size=(height, width)
        )

    def count_living_neighbors(self, r, c):
        """
        Counts the number of living neighbors around an array cell.

        Parameters
        ----------
            board : 2d array
            r : int
                row index
            c : int
                column index

        Returns
        ----------
            int
                number of living neighbors

        """
        num_rows, num_cols = self.board.shape

        u_ = self.board[(r-1) % num_rows, c % num_cols]
        d_ = self.board[(r+1) % num_rows, c % num_cols]
        l_ = self.board[r % num_rows, (c-1) % num_cols]
        r_ = self.board[r % num_rows, (c+1) % num_cols]
        ul = self.board[(r-1) % num_rows, (c-1) % num_cols]
        ur = self.board[(r-1) % num_rows, (c+1) % num_cols]
        dl = self.board[(r+1) % num_rows, (c-1) % num_cols]
        dr = self.board[(r+1) % num_rows, (c+1) % num_cols]

        return (u_ + d_ + l_ + r_ + ul + ur + dl + dr)

    def compute_next_state(self, current_state, num_living_neighbors):
        """
        Computes the state of a cell in the next generation of a Game of Life.

        Parameters
        ----------
            current_state : int
            num_living_neighbors : int

        Returns
        ----------
            next_state : int

        """
        next_state = self.DEAD

        if (num_living_neighbors < 2) and (current_state == self.ALIVE):
            # Dies by under-population
            next_state = self.DEAD
        elif ( 2 <= num_living_neighbors <= 3) and (current_state == self.ALIVE):
            # Survives on to the next generation
            next_state = self.ALIVE
        elif (num_living_neighbors > 3) and (current_state == self.ALIVE):
            # Dies by over-population
            next_state = self.DEAD
        elif (num_living_neighbors == 3) and (current_state == self.DEAD):
            # Live cell is birthed 
            next_state = self.ALIVE

        return next_state

In [ ]:
simulation = GameOfLife()
simulation.start(height=10, width=10)
while True:
    simulation.step()
    if simulation.is_empty():
        simulation.restart(height=10, width=10)

TODO (Written 11/19/16):
* Implement some tests for our functions
* Is it possible to write a better array display function? Don't want it to be overly complex but it would be nice to see black/white squares instead of the array print output.

## References

1. https://en.wikipedia.org/wiki/Conway's_Game_of_Life

2. https://en.wikipedia.org/wiki/Modulo_operation